### Objetivo

Neste notebook é implementada a camada de interpretação do sistema de diagnóstico desenvolvido nas fases anteriores.

Os principais objetivos são:

- Integrar uma LLM pré-treinada (Azure OpenAI) para interpretação de resultados
- Gerar explicações em linguagem natural a partir das métricas Recall, F1-score e Accuracy
- Comparar modelo sem GA e com GA
- Aplicar técnicas de prompt engineering adequadas ao contexto médico
- Avaliar qualitativamente as interpretações geradas
- Garantir reprodutibilidade, mesmo sem acesso à LLM real
- Preparar a base conceitual para futura integração com dados textuais (Fase 3).

A LLM não realiza predições, não substitui o modelo e não executa diagnóstico clínico. Seu papel é exclusivamente interpretativo.

In [ ]:
import warnings
warnings.filterwarnings("ignore", module="sklearn")

### Importação de bibliotecas

In [ ]:
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import os
from openai import AzureOpenAI
from typing import Dict
from dotenv import load_dotenv

### Carregamento dos artefatos do modelo otimizado
Os artefatos foram gerados no Notebook 2 e persistidos para reutilização.

- metrics_ga.csv – métricas do modelo otimizado via Algoritmo Genético
- best_hyperparameters.json - hiperparâmetros obtidos pelo Algoritmo Genético

In [ ]:
ARTIFACTS_PATH = Path("B:/Pós/hospital-ai-diagnosis/artefatos/")

with open(ARTIFACTS_PATH / "best_hyperparameters.json") as f:
    best_hyperparameters = json.load(f)

metrics = pd.read_csv(ARTIFACTS_PATH / "metrics.csv").iloc[0].to_dict()

metrics_ga = pd.read_csv(ARTIFACTS_PATH / "metrics_ga.csv").iloc[0].to_dict()

### Execução com ou sem LLM real

Para garantir a reprodutibilidade do experimento em ambientes acadêmicos, este notebook foi projetado para executar normalmente mesmo na ausência de credenciais do Azure OpenAI.

Quando as variáveis de ambiente não estão configuradas, utiliza-se um modo simulado, preservando o fluxo experimental e permitindo a avaliação metodológica da integração com LLMs.

### Papel da LLM no sistema

A LLM atua como uma camada de interpretação clínica, com as seguintes restrições:

- Não realiza diagnóstico
- Não substitui profissionais da saúde
- Baseia-se apenas nas métricas fornecidas
- Comunica limitações e incertezas do modelo


### Integração com Azure OpenAI

A interpretação dos resultados é realizada utilizando o serviço
Azure OpenAI, garantindo conformidade com padrões corporativos
de segurança, governança e auditoria.

A LLM é utilizada exclusivamente como camada de explicação
semântica, não interferindo no pipeline de diagnóstico.

Para garantir reprodutibilidade acadêmica, este notebook foi projetado para funcionar em dois modos:

- Modo real: utiliza Azure OpenAI, caso as variáveis de ambiente estejam configuradas;
- Modo simulado (mock): utilizado automaticamente quando as credenciais não estão disponíveis.

As credenciais não são versionadas e devem ser definidas localmente via arquivo .env.

In [ ]:
load_dotenv()


AZURE_VARS = {
"API_KEY": os.getenv("AZURE_OPENAI_API_KEY"),
"ENDPOINT": os.getenv("AZURE_OPENAI_ENDPOINT"),
"DEPLOYMENT": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
"API_VERSION": os.getenv("AZURE_API_VERSION")
}


USE_LLM = all(AZURE_VARS.values())


if USE_LLM:
    print("✅ Azure OpenAI configurado. Usando LLM real.")
else:
    print("⚠️ Azure OpenAI não configurado. Executando em modo simulado (mock).")

### Chamada à Azure OpenAI (execução real quando disponível, baseado nos pre-requisitos)

In [ ]:
def call_llm(prompt: str) -> str:
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_API_VERSION")

    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=subscription_key,
    )
    response = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "system",
                "content": (
                    "Você é um assistente especializado em interpretar resultados "
                    "de modelos de aprendizado de máquina aplicados à saúde. "
                    "Não forneça diagnósticos médicos e não extrapole além das métricas fornecidas."
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_completion_tokens=700
    )

    return response.choices[0].message.content

### Fallback local (execução garantida em qualquer ambiente)
Caso as credenciais não estejam configuradas, o notebook gera automaticamente uma explicação local coerente.

In [ ]:
def fallback_explanation(prompt: str) -> str:
    return (
        "O modelo otimizado por Algoritmo Genético apresentou maior recall, indicando maior capacidade de identificar corretamente pacientes com diabetes. "
        "Essa característica é particularmente relevante em cenários de triagem clínica, onde falsos negativos representam risco elevado.\n\n"
        "Observa-se, entretanto, que a priorização do recall pode resultar em trade-offs com outras métricas globais, como F1-score e acurácia, "
        "refletindo um possível aumento de falsos positivos. Esses resultados devem ser interpretados como suporte à decisão clínica, "
        "não substituindo a avaliação médica especializada."
    )

### Prompt Engineering para Interpretação Clínica

A integração com LLM não utiliza agentes autônomos.
O modelo de linguagem é empregado exclusivamente como uma
camada de interpretação e comunicação dos resultados técnicos.

O prompt foi projetado utilizando as seguintes técnicas de prompt engineering:

- Definição explícita de papel (role prompting);
- Injeção de contexto técnico (métricas e hiperparâmetros);
- Restrições claras de escopo e responsabilidade;
- Adequação da linguagem ao público-alvo (profissionais da saúde).

Essa abordagem garante respostas consistentes, controladas
e alinhadas ao domínio clínico, sem delegar tomada de decisão à LLM.

In [ ]:
def build_prompt(metrics_before, metrics_after, hyperparams):
    return f"""
Você é um assistente especializado em interpretar resultados de modelos de aprendizado de máquina aplicados à saúde.

Contexto:
- Problema: Classificação binária de diabetes
- Dataset rotulado previamente (avaliação offline)
- Modelo base: Regressão Logística
- Modelo otimizado via Algoritmo Genético, priorizando Recall

Hiperparâmetros selecionados pelo Algoritmo Genético:
{hyperparams}

Métricas do modelo sem GA:
{metrics_before}

Métricas do modelo com GA:
{metrics_after}

Instruções:
- Explique como a otimização de hiperparâmetros pode ter impactado as métricas
- Destaque o papel do class_weight balanceado no contexto médico
- Relacione Recall elevado com redução de falsos negativos
- Não forneça diagnóstico médico
- Não extrapole além dos dados apresentados
- Máximo de 3 parágrafos
"""

### Execução unificada (LLM ou fallback automático)

In [ ]:
prompt = build_prompt(
    metrics,
    metrics_ga,
    best_hyperparameters
)

interpretation = call_llm(prompt)
print(interpretation)

### Avaliação Qualitativa das Interpretações

A avaliação da qualidade das interpretações geradas foi realizada de forma qualitativa, considerando os seguintes critérios:

|Critério|	Avaliação|
|--|--|
|Clareza da linguagem|	Alta|
|Fidelidade às métricas|	Alta|
|Adequação ao contexto médico|	Alta
|Ausência de extrapolação clínica|	Satisfatória
|Utilidade como apoio à decisão|	Moderada a Alta

Observou-se que a estruturação adequada do prompt reduz significativamente o risco de interpretações imprecisas ou especulativas.

### Conclusão

Este notebook demonstrou uma abordagem responsável e reprodutível para a integração de LLMs na interpretação de resultados de modelos de Machine Learning aplicados à saúde.   
A utilização de prompt engineering, aliada a mecanismos de fallback, garante clareza metodológica, segurança e adequação ao contexto acadêmico, atendendo plenamente aos requisitos da Fase 2 do projeto.